In [1]:
# import necessary libraries

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

# BigQuery settings
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
# ignore if local tmp.csv already exists in the time_mapping folder. We need this to identify the dates
# read the time matching CSV and remove MRNs for security reasons (ignore if local tmp.csv already exists)
import pandas as pd
data_frame = pd.read_csv('time_mapping/jon_mapping.csv')
data_frame = data_frame.drop('MRN', axis = 1) 
data_frame.to_csv('time_mapping/tmp.csv', index = False)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Create a table of ADT of cohort patients (with jittered times) (ADT_cohort_jit) in order to find all the patients at the same department during the diagnostic process**

*Skip if table ADT_cohort_jit already exists on GCP*

**Remark:** time_out is either the actual time_out or TPA_admin_time, whichever is earlier

In [10]:

query =  """
drop table if exists noshad.ADT_cohort_jit;
create table noshad.ADT_cohort_jit as
(
SELECT ADT.jc_uid, ADT.pat_enc_csn_id_coded, ADT.department_id, CH.tpaAdminTime,
    min(ADT.event_time_jittered) AS time_in, max(ADT.event_time_jittered) AS time_out
FROM `starr_datalake2018.adt` AS ADT
INNER JOIN `noshad.cohort_v2` AS CH
  USING (pat_enc_csn_id_coded)
GROUP BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, ADT.department_id, CH.tpaAdminTime
ORDER BY ADT.jc_uid, ADT.pat_enc_csn_id_coded, time_in
)
"""
cursor.execute(query);

#results = cursor.fetchall();
#print(results[:2])
#results_np = np.array(results)

## Extract Feature: Provider Busyness all_actions

In [21]:

# CREATE OR REPLACE TABLE noshad.Feature_prov_busy_clinical_actions


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'time_mapping/tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')


query= """

CREATE OR REPLACE TABLE noshad.Feature_prov_busy_all_actions as (
WITH

    -- Generate cohort table with actual times
  CHT_real_date AS
  (SELECT CHT.* except(tpaAdminTime), 
  DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime,
  DATETIME_SUB(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_before,
  DATETIME_ADD(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_after,
  EXTRACT(DATE FROM DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY)) as tpa_date
  
  FROM `noshad.cohort_v2` as CHT,
  `noshad.tmp` as TMP
  
  WHERE CHT.jc_uid=TMP.ANON_ID
  
  ),
  
  
    -- Generate AL with actual times
  AL_real_date AS
  (SELECT AL.*, 
      DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real,
      EXTRACT(DATE FROM DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY)) as access_date
  FROM `noshad.shc_access_log_de_dep_id` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
    AND AL.department_id = 2001002
    AND 
    (
    metric_name like "%Lab%" OR metric_name like "%lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Report%" OR metric_name like "%report%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Medications%" OR metric_name like "%Medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"

    )
  ),
  
  Team_users as
  (
  SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(*) as user_cohort_actions
  FROM  CHT_real_date INNER JOIN AL_real_date ON (CHT_real_date.pat_enc_csn_id_coded= AL_real_date.csn)
  WHERE AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
  GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid 
  ),
  
  Prov_busy as ( 
  
    SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(*) as user_all_actions, 
    FROM  CHT_real_date INNER JOIN AL_real_date on (tpa_date=access_date)
    WHERE 
        AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
    GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid
    )

SELECT Team_users.jc_uid, Team_users.pat_enc_csn_id_coded, avg(Prov_busy.user_all_actions) as prov_busy_all_actions, count(Team_users.user_cohort_actions) as team_size
    FROM  Team_users LEFT JOIN Prov_busy USING (pat_enc_csn_id_coded)
    WHERE Team_users.user_deid = Prov_busy.user_deid
    GROUP BY jc_uid, pat_enc_csn_id_coded
)

"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
feature extraced
temporary tmp file deleted


## Extract Feature: Provider Busyness specific_actions

In [25]:

# CREATE OR REPLACE TABLE noshad.Feature_prov_busy_specific_actions


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'time_mapping/tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')


query= """

CREATE OR REPLACE TABLE noshad.Feature_prov_busy_specific_actions as (
WITH

    -- Generate cohort table with actual times
  CHT_real_date AS
  (SELECT CHT.* except(tpaAdminTime), 
  DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime,
  DATETIME_SUB(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_before,
  DATETIME_ADD(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_after,
  EXTRACT(DATE FROM DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY)) as tpa_date
  
  FROM `noshad.cohort_v2` as CHT,
  `noshad.tmp` as TMP
  
  WHERE CHT.jc_uid=TMP.ANON_ID
  
  ),
  
  
    -- Generate AL with actual times
  AL_real_date AS
  (SELECT AL.*, 
      DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real,
      EXTRACT(DATE FROM DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY)) as access_date
  FROM `noshad.shc_access_log_de_dep_id` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
    AND AL.department_id = 2001002
    AND 
    (
    --metric_name like "%Lab%" OR metric_name like "%lab%"
    --OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    --OR metric_name like "%Report%" OR metric_name like "%report%"
    --OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Result%" OR metric_name like "%result%"  
    --OR metric_name like "%Medications%" OR metric_name like "%Medications%"
    --OR metric_name like "%MAR%" 
    --OR metric_name like "%Note%" OR metric_name like "%note%"
    --OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"

    )
  ),
  
  Team_users as
  (
  SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(*) as user_cohort_actions
  FROM  CHT_real_date INNER JOIN AL_real_date ON (CHT_real_date.pat_enc_csn_id_coded= AL_real_date.csn)
  WHERE AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
  GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid 
  ),
  
  Prov_busy as ( 
  
    SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(*) as user_all_actions, 
    FROM  CHT_real_date INNER JOIN AL_real_date on (tpa_date=access_date)
    WHERE 
        AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
    GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid
    )

SELECT Team_users.jc_uid, Team_users.pat_enc_csn_id_coded, avg(Prov_busy.user_all_actions) as prov_busy_specific_actions, count(Team_users.user_cohort_actions) as team_size
    FROM  Team_users LEFT JOIN Prov_busy USING (pat_enc_csn_id_coded)
    WHERE Team_users.user_deid = Prov_busy.user_deid
    GROUP BY jc_uid, pat_enc_csn_id_coded
)

"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
feature extraced
temporary tmp file deleted


In [26]:
## Extract Feature: Provider Busyness num patients

In [28]:

# CREATE OR REPLACE TABLE noshad.Feature_prov_busy_num_pat


client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();

# Upload time_mapping tmp.CSV 

schemafield_col1 = bigquery.schema.SchemaField("ANON_ID","STRING") #Define your schema
schemafield_col2 = bigquery.schema.SchemaField("JITTER","INTEGER")

filename = 'time_mapping/tmp.csv'
table_id = 'tmp' # the name of the chart to create

dataset_ref = client.dataset('noshad')
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig()
job_config.source_format = bigquery.SourceFormat.CSV
job_config.skip_leading_rows = 1
job_config.autodetect = True

with open(filename, "rb") as source_file:
    job = client.load_table_from_file(source_file, table_ref, job_config=job_config)

job.result()  # Waits for table load to complete.

print('temporary tmp file generated')


query= """

CREATE OR REPLACE TABLE noshad.Feature_prov_busy_num_pat as (
WITH

    -- Generate cohort table with actual times
  CHT_real_date AS
  (SELECT CHT.* except(tpaAdminTime), 
  DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY) as tpaAdminTime,
  DATETIME_SUB(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_before,
  DATETIME_ADD(DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY), INTERVAL 60 MINUTE) as tpa_60_after,
  EXTRACT(DATE FROM DATETIME_SUB(CHT.tpaAdminTime, INTERVAL TMP.JITTER DAY)) as tpa_date
  
  FROM `noshad.cohort_v2` as CHT,
  `noshad.tmp` as TMP
  
  WHERE CHT.jc_uid=TMP.ANON_ID
  
  ),
  
  
    -- Generate AL with actual times
  AL_real_date AS
  (SELECT AL.*, 
      DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY) as access_time_real,
      EXTRACT(DATE FROM DATETIME_SUB(AL.access_time_jittered, INTERVAL TMP.JITTER DAY)) as access_date
  FROM `noshad.shc_access_log_de_dep_id` as AL,
  `noshad.tmp` as TMP
  WHERE AL.rit_uid=TMP.ANON_ID
    AND AL.department_id = 2001002
    AND 
    (
    metric_name like "%Lab%" OR metric_name like "%lab%"
    OR metric_name like "%Encounter%" OR metric_name like "%encounter%"
    OR metric_name like "%Report%" OR metric_name like "%report%"
    OR metric_name like "%Flowchart%" OR metric_name like "%flowchart%"
    OR metric_name like "%Order%" OR metric_name like "%order%"   
    OR metric_name like "%Result%" OR metric_name like "%result%"  
    OR metric_name like "%Medications%" OR metric_name like "%Medications%"
    OR metric_name like "%MAR%" 
    OR metric_name like "%Note%" OR metric_name like "%note%"
    OR metric_name like "%History%" OR metric_name like "%history%"
    OR metric_name like "%Imag%" OR metric_name like "%imag%"

    )
  ),
  
  -- Find the clinical users associated with a patient
  Team_users as
  (
  SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(*) as user_cohort_actions
  FROM  CHT_real_date INNER JOIN AL_real_date ON (CHT_real_date.pat_enc_csn_id_coded= AL_real_date.csn)
  WHERE AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
  GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid 
  ),
  
  -- Find the provider busyness for all patients and all providers during the TPA admin time (60 min before and after)
  Prov_busy as ( 
  
    SELECT CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid, count(DISTINCT AL_real_date.rit_uid) as num_pat, 
    FROM  CHT_real_date INNER JOIN AL_real_date on (tpa_date=access_date)
    WHERE 
        AL_real_date.access_time_real BETWEEN CHT_real_date.tpa_60_before AND CHT_real_date.tpa_60_after 
    GROUP By CHT_real_date.jc_uid, CHT_real_date.pat_enc_csn_id_coded, AL_real_date.user_deid
    )

SELECT Team_users.jc_uid, Team_users.pat_enc_csn_id_coded, avg(Prov_busy.num_pat) as prov_busy_num_pat, count(Team_users.user_cohort_actions) as team_size
    FROM  Team_users LEFT JOIN Prov_busy USING (pat_enc_csn_id_coded)
    WHERE Team_users.user_deid = Prov_busy.user_deid
    GROUP BY jc_uid, pat_enc_csn_id_coded
)

"""

cursor.execute(query);

print('feature extraced')

## Final step: delete the temporary time mapping
query = "DROP TABLE noshad.tmp"
cursor.execute(query);

print('temporary tmp file deleted')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


temporary tmp file generated
feature extraced
temporary tmp file deleted
